In [ ]:
import torch
from transformers import AdamW, BertTokenizerFast, BertForQuestionAnswering

<h3> Install transformers </h3>

In [ ]:
!pip install packaging==21.3

In [ ]:
!pip install transformers==4.5.0

  Using cached transformers-4.5.0-py3-none-any.whl (2.1 MB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


<h3> Load Model and Tokenizer </h3>


In [ ]:
# model_name can be one of models in huggingface model hub

model_name = 'bert-base-uncased'
model = BertForQuestionAnswering.from_pretrained(model_name)
eng_tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Import Packages

In [ ]:
import math
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = torch.device("cuda", 1) if torch.cuda.is_available() else "cpu"


def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(0)    # Fix random seed for reproducibility

<h1> -> Task:</h2>

In [ ]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)

fp16_training = True

if fp16_training:
    %pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 877.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

## Load Model and Tokenizer






In [ ]:
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")





Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def convert_json(input_file, output_file):
    # Load the original JSON file
    with open(input_file, 'r') as f:
        original_data = json.load(f)

    # Initialize lists to store questions and paragraphs
    questions = []
    paragraphs = []

    # Extract data from original JSON and restructure it
    for data_item in original_data['data']:
        for paragraph_data in data_item['paragraphs']:
            paragraph_text = paragraph_data['context']
            paragraphs.append(paragraph_text)
            for qa in paragraph_data['qas']:
                question_id = len(questions)
                answer_start = qa['answers'][0]['answer_start']
                answer_end = answer_start + len(qa['answers'][0]['text'])
                question_text = qa['question']
                answer_text = qa['answers'][0]['text']
                questions.append({
                    "id": question_id,
                    "paragraph_id": len(paragraphs) - 1,
                    "question_text": question_text,
                    "answer_text": answer_text,
                    "answer_start": answer_start,
                    "answer_end": answer_end
                })

    # Create the new JSON structure
    new_json = {
        "questions": questions,
        "paragraphs": paragraphs
    }

    # Save the new JSON to a file
    with open(output_file, 'w') as f:
        json.dump(new_json, f, indent=4, ensure_ascii=False)

# Example usage:
convert_json('spoken_train-v1.1.json', 'converted_train.json')
convert_json('spoken_test-v1.1_WER54.json', 'converted_dev.json')
convert_json('spoken_test-v1.1.json', 'converted_test.json')


In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("converted_train.json")
dev_questions, dev_paragraphs     = read_data("converted_dev.json")
test_questions, test_paragraphs   = read_data("converted_test.json")

print('train_questions:', len(train_questions))
print('validation_questions:', len(dev_questions))
print('test_questions:', len(test_questions))

train_questions: 37111
validation_questions: 5351
test_questions: 5351


## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately
train_questions_tokenized  = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized    = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized   = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized   = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized  = tokenizer(test_paragraphs, add_special_tokens=False)



Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


## Dataset and Dataloader

In [ ]:
DOC_STRIDE = None

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 350

        ##### Change value of doc_stride #####
        self.doc_stride = int(0.9 * self.max_paragraph_len)

        ############################################
        global DOC_STRIDE

        DOC_STRIDE = self.doc_stride
        ############################################
        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]

        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### Preprocessing #####

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])
            if answer_start_token is None or answer_end_token is None:
              answer_start_token = 0
              answer_end_token = 0

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            prefix_len = int(random.random() * self.max_paragraph_len)
            postfix_len = self.max_paragraph_len - prefix_len
            paragraph_start, paragraph_end = mid - prefix_len, mid + postfix_len
            if paragraph_start < 0:
                paragraph_end -= paragraph_start
                paragraph_start = 0
            if paragraph_end >= len(tokenized_paragraph):
                paragraph_end = len(tokenized_paragraph) - 1

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]

            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation
        # Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []

            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):

                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]

                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)

                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)

            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len

        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4

#########################

train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader   = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader  = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [ ]:
def evaluate(data, output, paragraph, paragraph_tokenized):
    ##### Postprocessing #####

    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]

    paragraph_start_index = 0
    paragraph_end_index = 0

    for k in range(num_of_windows):

        mask = (data[1][0][k].bool() & data[2][0][k].bool()).to(device)

        masked_output_start = torch.masked_select(output.start_logits[k], mask)
        masked_output_start = masked_output_start[:-1]

        start_prob, start_index = torch.max(masked_output_start, dim=0)

        masked_output_end = torch.masked_select(output.end_logits[k], mask)
        masked_output_end = masked_output_end[start_index: -1]

        end_prob, end_index = torch.max(masked_output_end, dim=0)

        end_index += start_index

        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        masked_data = torch.masked_select(data[0][0][k].to(device), mask)[:-1]

        # Replace answer if calculated probability is larger than previous windows
        if (prob > max_prob) and (start_index <= end_index <= (start_index + 50)):
            max_prob = prob
            paragraph_start_index = start_index.item() + (DOC_STRIDE * k)
            paragraph_end_index = end_index.item() + (DOC_STRIDE * k)
            answer = tokenizer.decode(masked_data[start_index : end_index + 1])



    ##########
    char_count = 0
    start_flag = False

    for i, token in enumerate(paragraph_tokenized):
        if token in ('[UNK]', '[CLS]', '[SEP]'):
            if i == paragraph_start_index:
                new_start = char_count
            if i == paragraph_end_index:
                new_end = char_count
            char_count += 1
        else:
            for char in token:
                if i == paragraph_start_index and not start_flag:
                    new_start = char_count
                    start_flag = True
                if i == paragraph_end_index:
                    new_end = char_count
                if char == "#":
                    continue
                else:
                    while char_count < len(paragraph) and char != paragraph[char_count]:
                        char_count += 1
                    char_count += 1

    if "[UNK]" in answer:
        print(f"original answer: {answer}")
        answer = paragraph[new_start: new_end+1]
        print(f"corrected answer: {answer}")
        print("-"*50)

###########################################################

    answer = answer.replace(' ', '')

###########################################################

    if len(answer) > 1:
        if "「" not in answer and answer[-1] == "」":
            answer = answer[:-1]
    return answer

## Training

In [ ]:
num_epoch     = 5
validation    = True
logging_step  = 500
learning_rate = 5e-6

optimizer = AdamW(model.parameters(), lr=learning_rate)

##### Apply linear learning rate decay #####
total_steps = len(train_loader) * num_epoch
# warmup_steps = int(0.05 * total_steps)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

##################################################

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for batch_idx, data in enumerate(tqdm(train_loader)):
        # Load all data into GPU
        data = [i.to(device) for i in data]

        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)

        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss

        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()

        ##### Apply linear learning rate decay #####
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        step += 1
        ##################################################

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1}  Step {step}  Loss = {train_loss.item() / logging_step:.3f}, Accuracy = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, dev_paragraphs[dev_questions[i]['paragraph_id']], dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].tokens) == dev_questions[i]["answer_text"]
            print(f"Validation  Epoch {epoch + 1}  Accuracy = {dev_acc / len(dev_loader):.3f}")
        model.train()


##################################################

# Save a model
print("Saving Model ...")
model_save_dir = "./models/saved_model"
model.save_pretrained(model_save_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Start Training ...


  0%|          | 0/9278 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1  Step 500  Loss = 2.464, Accuracy = 0.562
Epoch 1  Step 1000  Loss = 1.206, Accuracy = 0.782
Epoch 1  Step 1500  Loss = 1.064, Accuracy = 0.802
Epoch 1  Step 2000  Loss = 1.080, Accuracy = 0.782
Epoch 1  Step 2500  Loss = 1.011, Accuracy = 0.779
Epoch 1  Step 3000  Loss = 1.004, Accuracy = 0.772
Epoch 1  Step 3500  Loss = 0.990, Accuracy = 0.775
Epoch 1  Step 4000  Loss = 0.949, Accuracy = 0.780
Epoch 1  Step 4500  Loss = 0.973, Accuracy = 0.767
Epoch 1  Step 5000  Loss = 0.958, Accuracy = 0.765
Epoch 1  Step 5500  Loss = 0.841, Accuracy = 0.793
Epoch 1  Step 6000  Loss = 0.843, Accuracy = 0.775
Epoch 1  Step 6500  Loss = 0.907, Accuracy = 0.760
Epoch 1  Step 7000  Loss = 0.833, Accuracy = 0.780
Epoch 1  Step 7500  Loss = 0.804, Accuracy = 0.775
Epoch 1  Step 8000  Loss = 0.760, Accuracy = 0.794
Epoch 1  Step 8500  Loss = 0.748, Accuracy = 0.793
Epoch 1  Step 9000  Loss = 0.766, Accuracy = 0.779
Evaluating Dev Set ...


...

Validation  Epoch 4  Accuracy = 0.001


  0%|          | 0/9278 [00:00<?, ?it/s]

Epoch 5  Step 500  Loss = 0.371, Accuracy = 0.871
Epoch 5  Step 1000  Loss = 0.362, Accuracy = 0.869
Epoch 5  Step 1500  Loss = 0.367, Accuracy = 0.874
Epoch 5  Step 2000  Loss = 0.374, Accuracy = 0.871
Epoch 5  Step 2500  Loss = 0.327, Accuracy = 0.874
Epoch 5  Step 3000  Loss = 0.394, Accuracy = 0.859
Epoch 5  Step 3500  Loss = 0.356, Accuracy = 0.878
Epoch 5  Step 4000  Loss = 0.381, Accuracy = 0.865
Epoch 5  Step 4500  Loss = 0.362, Accuracy = 0.867
Epoch 5  Step 5000  Loss = 0.364, Accuracy = 0.874
Epoch 5  Step 5500  Loss = 0.353, Accuracy = 0.875
Epoch 5  Step 6000  Loss = 0.367, Accuracy = 0.876
Epoch 5  Step 6500  Loss = 0.342, Accuracy = 0.872
Epoch 5  Step 7000  Loss = 0.382, Accuracy = 0.868
Epoch 5  Step 7500  Loss = 0.391, Accuracy = 0.855
Epoch 5  Step 8000  Loss = 0.388, Accuracy = 0.859
Epoch 5  Step 8500  Loss = 0.406, Accuracy = 0.858
Epoch 5  Step 9000  Loss = 0.392, Accuracy = 0.864
Evaluating Dev Set ...


  0%|          | 0/5351 [00:00<?, ?it/s]

Validation  Epoch 5  Accuracy = 0.001
Saving Model ...


## View Results

In [ ]:
model.eval()
with torch.no_grad():
    dev_acc = 0
    for i, data in enumerate(tqdm(dev_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
               attention_mask=data[2].squeeze(dim=0).to(device))
        # prediction is correct only if answer text exactly matches
        pred_answer = evaluate(data, output, dev_paragraphs[dev_questions[i]['paragraph_id']], dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].tokens)
        true_answer = dev_questions[i]["answer_text"]
        dev_acc += (pred_answer == true_answer)
        if pred_answer != true_answer:
            print("-"*50)
            print(f"Correct Answer: {true_answer}")
            print(f"Predict Answer: {pred_answer}")
            print("-"*50)

model.train()
print(f"Validation  Accuracy = {dev_acc / len(dev_loader):.3f}")



  0%|          | 0/5351 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
****************************************
****************************************
Correct Answer: Alpha Phi Omega
Predict Answer: there
****************************************
****************************************
Correct Answer: ten
Predict Answer: there
****************************************
****************************************
Correct Answer: May
Predict Answer: every
****************************************
****************************************
Correct Answer: Summer Breeze
Predict Answer: every
****************************************
****************************************
Correct Answer: Larry Ellison
Predict Answer: even
****************************************
****************************************
Correct Answer: Larry Ellison
Predict Answer: even
****************************************
****************************************
Correct Answer: David Axelrod
Predict Answer: known
*******************************

## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, test_paragraphs[test_questions[i]['paragraph_id']], test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "./result.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/5351 [00:00<?, ?it/s]

Completed! Result is in ./result.csv
